In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import gensim
import pickle
# DataFrame
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences


In [5]:
# Loading individual arrays
x_test = np.load('input/x_test.npy')
y_test = np.load('input/y_test.npy')

In [23]:
BATCH_SIZE = 1024
SEQUENCE_LENGTH = 300
DATASET_ENCODING = "ISO-8859-1"


In [12]:
# Define file paths (you should use the same paths that you used for saving)
model_folder = 'model/base/'
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

# Load the Keras model
model = load_model(model_folder + KERAS_MODEL)

# Load the Word2Vec model
w2v_model = gensim.models.Word2Vec.load(model_folder + WORD2VEC_MODEL)

# Load the Tokenizer
with open(model_folder + TOKENIZER_MODEL, 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the Label Encoder
with open(model_folder + ENCODER_MODEL, 'rb') as handle:
    encoder = pickle.load(handle)

# Print summary to confirm model loaded correctly
model.summary()

C:\Users\kwfen\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (1024, 300, 100)            │      29,041,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (1024, 300, 100)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (1024, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (1024, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,122,403 (111.09 MB)

 Trainable params: 80,501 (314.46 KB)

 Non-trainable params: 29,041,900 (110.79 MB)

 Optimizer params: 2 (12.00 B)

In [13]:
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

313/313 ━━━━━━━━━━━━━━━━━━━━ 57s 182ms/step - accuracy: 0.7838 - loss: 0.4545

ACCURACY: 0.7832156419754028
LOSS: 0.4557904303073883


In [19]:
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

In [20]:
import logging
import time
# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)
predict("I love the music")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


{'label': 'POSITIVE',
 'score': 0.9778547286987305,
 'elapsed_time': 0.159590482711792}

In [24]:
journal_file = '../../archive/100_journals.csv'
df_journal = pd.read_csv(journal_file, encoding=DATASET_ENCODING)
df_journal['text'] = df_journal['journal']

In [25]:
x_test = pad_sequences(tokenizer.texts_to_sequences(df_journal.text), maxlen=SEQUENCE_LENGTH)

In [27]:
# Make predictions
predictions = model.predict(x_test)

# Assuming it's a binary classification, convert probabilities to class labels
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Convert the predicted classes to their original label format using the encoder
original_labels = encoder.inverse_transform(predicted_classes)

# Add the predictions to the dataframe
df_journal['predictions'] = original_labels

# Save the predictions to a new CSV file if needed
df_journal.to_csv('path_to_save_predictions.csv', index=False)

# Print the first few rows of the dataframe to verify
print(df_journal.head())

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
   age            topic                                            journal  \
0   15  Swimming Lesson  Swimming lesson was today. I learned a new str...   
1   15        Christmas  Christmas was excited! I got a lot of gifts an...   
2   13      Babysitting  I babysat my little brother today. It was depr...   
3   15     Science Fair  Today was the science fair. My project was abo...   
4    9    Library Visit  I visited the library today and got new books ...   

   score                                               text predictions  
0      8  Swimming lesson was today. I learned a new str...    NEGATIVE  
1     88  Christmas was excited! I got a lot of gifts an...    POSITIVE  
2      1  I babysat my little brother today. It was depr...    NEGATIVE  
3      5  Today was the science fair. My project was abo...    NEGATIVE  
4     10  I visited the library today and got new books ...    NEGATIVE  
